In [1]:
import os
import random
import torch
import numpy as np
from torch.nn import functional as F

dataset_dir = "./family"
all_trip_file = os.path.join(dataset_dir, "all.txt")
relations_file = os.path.join(dataset_dir, "relations.txt")
entities_file = os.path.join(dataset_dir, "entities.txt")

In [2]:
device = "cuda"

In [3]:
def read_xxx_to_id(file_path, num_read=None):
    xxx2id = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if num_read:
            lines = random.sample(lines, num_read)
        for line in lines:
            line = line.strip()
            xxx2id[line] = len(xxx2id)
    return xxx2id

def parse_triplets(triplets_file: str,
                   rel2id: dict,
                   ent2id: dict):
    """Read triplets (relation, head, tail)."""
    triplets = []
    with open(triplets_file, 'r') as file:
        for line in file:
            line = line.strip().split('\t')
            assert(len(line) == 3)
            try:
                triplets.append(
                    (
                        rel2id[line[1]],
                        ent2id[line[0]],
                        ent2id[line[2]]
                    )
                )
            except KeyError:
                pass
    return triplets

## Read file and resample

In [4]:
num_entities = None
num_relations = None

In [5]:
rel2id = read_xxx_to_id(relations_file, num_relations)
id2rel = {ident: rel for rel, ident in rel2id.items()}

ent2id = read_xxx_to_id(entities_file, num_entities)
id2ent = {ident: ent for ent, ident in ent2id.items()}

all_facts = parse_triplets(all_trip_file, rel2id, ent2id)
# relation to (head, tail)
rel2ht = {rel: [] for rel in id2rel.keys()}
for (r, h, t) in all_facts:
    rel2ht[r].append((h, t))

num_rel, num_ent, num_trip = len(rel2id), len(ent2id), len(all_facts)

num_rel, num_ent, num_trip

(12, 3007, 28356)

## Compute bifurcation

In [6]:
from collections import defaultdict

def get_head_bifur(head_tail_pairs: list, k: int):
    bifur_cnt = 0
    head2tails = defaultdict(list)
    for h, t in head_tail_pairs:
        head2tails[h].append(t)
        if len(head2tails[h]) == k:
            bifur_cnt += 1
    return head2tails, bifur_cnt / len(head2tails.keys())

def get_tail_bifur(head_tail_pairs: list, k: int):
    bifur_cnt = 0
    tail2heads = defaultdict(list)
    for h, t in head_tail_pairs:
        tail2heads[t].append(h)
        if len(tail2heads[t]) == k:
            bifur_cnt += 1
    return tail2heads, bifur_cnt / len(tail2heads.keys())

In [7]:
max_lambda = 7
backward = False

for rel, pairs in rel2ht.items():
    print(f"[{id2rel[rel]}]")
    for k in range(2, max_lambda + 1):
        head2tails, bifur_head = get_head_bifur(pairs, k)
        print(f"bifur_head({k}): {bifur_head:.2f}")
        if backward:
            tail2heads, bifur_tail = get_tail_bifur(pairs, k)
            print(f"bifur_tail({k}): {bifur_tail:.2f}")
    print("\n")

[aunt]
bifur_head(2): 0.85
bifur_head(3): 0.74
bifur_head(4): 0.65
bifur_head(5): 0.54
bifur_head(6): 0.49
bifur_head(7): 0.44


[brother]
bifur_head(2): 0.77
bifur_head(3): 0.57
bifur_head(4): 0.42
bifur_head(5): 0.30
bifur_head(6): 0.23
bifur_head(7): 0.18


[daughter]
bifur_head(2): 0.84
bifur_head(3): 0.00
bifur_head(4): 0.00
bifur_head(5): 0.00
bifur_head(6): 0.00
bifur_head(7): 0.00


[father]
bifur_head(2): 0.42
bifur_head(3): 0.26
bifur_head(4): 0.17
bifur_head(5): 0.12
bifur_head(6): 0.07
bifur_head(7): 0.05


[husband]
bifur_head(2): 0.14
bifur_head(3): 0.02
bifur_head(4): 0.01
bifur_head(5): 0.00
bifur_head(6): 0.00
bifur_head(7): 0.00


[mother]
bifur_head(2): 0.53
bifur_head(3): 0.35
bifur_head(4): 0.22
bifur_head(5): 0.14
bifur_head(6): 0.08
bifur_head(7): 0.06


[nephew]
bifur_head(2): 0.82
bifur_head(3): 0.70
bifur_head(4): 0.55
bifur_head(5): 0.43
bifur_head(6): 0.34
bifur_head(7): 0.28


[niece]
bifur_head(2): 0.87
bifur_head(3): 0.73
bifur_head(4): 0.60
bifur_head(5)

## Compute macro, micro and comprehensive saturation

In [8]:
def get_adjacency_matrices(triplets,
                           num_relations: int,
                           num_entities: int
                           ):
    """Compute adjacency matrix from all triplets
       in preparation for creating sparse matrix in torch.
    """
    matrix = {
        r: ([[0, 0]], [0.], [num_entities, num_entities])
        for r in range(num_relations)
    }
    for triplet in triplets:
        rel = triplet[0]
        head = triplet[1]
        tail = triplet[2]
        value = 1.
        matrix[rel][0].append([head, tail])
        matrix[rel][1].append(value)

    for rel, mat in matrix.items():
        matrix[rel] = torch.sparse.FloatTensor(
            torch.LongTensor(mat[0]).t(),
            torch.FloatTensor(mat[1]),
            mat[2]
        ).to(device)

    return matrix

# `adj_matrices`:  adjacency matrices, ORDER matters!!!
# `head_nodes`: head nodes list
# return: a list of `batch_size` nodes
def from_head_hops(adj_matrices: list,
                   head_nodes: list
                   ):
    # (batch_size, num_entities)
    v_x = F.one_hot(torch.LongTensor(head_nodes), adj_matrix[0].size(0)).float().to(device)
    # (num_entities, num_entities)
    result = torch.matmul(adj_matrices[0].t(), v_x.t())
    for mat in adj_matrices[1:]:
        result = torch.mm(mat.t(), result)
    # (batch_size, num_entites)
    result = result.t().cpu().numpy()
    indices = np.argwhere(result > 0)
    # {head: {tail: num_paths}}
    ret = {head: {} for head in head_nodes}
    for row, col in indices:
        # `row`: (row, column) ==> (head, tail)
        ret[head_nodes[row]][col] = result[row, col]
    return ret

In [9]:
adj_matrix = get_adjacency_matrices(all_facts, num_rel, num_ent)
adj_matrix[0], from_head_hops([adj_matrix[1], adj_matrix[2]], list(ent2id.values())[:2])

(tensor(indices=tensor([[   0, 2675,  267,  ..., 1973, 2096, 2097],
                        [   0, 2964,  369,  ..., 2023, 2866, 2866]]),
        values=tensor([0., 1., 1.,  ..., 1., 1., 1.]),
        device='cuda:0', size=(3007, 3007), nnz=2991, layout=torch.sparse_coo),
 {0: {}, 1: {0: 5.0, 1110: 5.0}})

In [10]:
from itertools import permutations

topk_macro = 10
topk_micro = 10
topk_comp = 10
max_rule_len = 2  # T
relations = list(id2rel.keys())
paths_permut = [(rel1, rel2) for rel1 in relations for rel2 in relations]

len(paths_permut)

144

In [11]:
from time import time
from collections import defaultdict

start = time()

macro_saturations = {rel: {path: 0. for path in paths_permut} for rel in id2rel.keys()}
tmp_micro_saturations = {rel: {path: {} for path in paths_permut} for rel in id2rel.keys()}  # {path: {(head, tail): num_paths}}
micro_saturations = {rel: {path: 0. for path in paths_permut} for rel in id2rel.keys()}
total_paths_pairs = {rel: defaultdict(int) for rel in id2rel.keys()}  # {(head, tail): num_total_paths}
# get number of triplets under each relation
num_rel2trip = {rel: len(rel2ht[rel]) for rel in id2rel.keys()}
# get triplets under each relation
rel_head2tails = {rel: defaultdict(list) for rel in id2rel.keys()}
for (r, h, t) in all_facts:
    rel_head2tails[r][h].append(t)
    
for rel in rel_head2tails:
    for path in macro_saturations[rel].keys():
        matrices = [adj_matrix[r] for r in path]
        heads = list(rel_head2tails[rel].keys())
        num_paths_from_heads = from_head_hops(matrices, heads)
        for head, tails in rel_head2tails[rel].items():
            for tail in tails:
                if tail in num_paths_from_heads[head]:
                    macro_saturations[rel][path] += 1.
                    tmp_micro_saturations[rel][path][(head, tail)] = num_paths_from_heads[head][tail]
                    total_paths_pairs[rel][(head, tail)] += num_paths_from_heads[head][tail]
        macro_saturations[rel][path] /= num_rel2trip[rel]
    for path, pairs in tmp_micro_saturations[rel].items():
        for pair, num_path in pairs.items():
            # `pair`: (head, tail)
            micro_saturations[rel][path] += num_path / total_paths_pairs[rel][pair]
        if len(tmp_micro_saturations[rel][path]) != 0:
            micro_saturations[rel][path] /= num_rel2trip[rel]

print(f"{time() - start}s")

26.830204486846924s


### Macro saturation

In [12]:
for rel in macro_saturations:
    print(f"{id2rel[rel]:=^50}")
    sorted_items = sorted(macro_saturations[rel].items(), key=lambda x: x[1], reverse=True)
    for i, (path, saturation) in enumerate(sorted_items):
#         if i == topk_macro:
        if i == 15:
            break
        print(f"{tuple(id2rel[r] for r in path)}: {saturation:.2f}")        
    print("\n")

=======================aunt=======================
('sister', 'uncle'): 0.89
('sister', 'aunt'): 0.85
('aunt', 'brother'): 0.83
('aunt', 'sister'): 0.75
('sister', 'father'): 0.66
('sister', 'mother'): 0.34
('aunt', 'husband'): 0.02
('aunt', 'wife'): 0.02
('wife', 'uncle'): 0.00
('mother', 'wife'): 0.00
('mother', 'husband'): 0.00
('mother', 'mother'): 0.00
('mother', 'uncle'): 0.00
('aunt', 'mother'): 0.00
('aunt', 'uncle'): 0.00


=====================brother======================
('son', 'father'): 1.00
('son', 'mother'): 0.98
('brother', 'brother'): 0.86
('brother', 'sister'): 0.81
('nephew', 'uncle'): 0.77
('nephew', 'aunt'): 0.68
('uncle', 'nephew'): 0.64
('uncle', 'niece'): 0.58
('father', 'nephew'): 0.33
('uncle', 'son'): 0.28
('father', 'niece'): 0.25
('uncle', 'daughter'): 0.22
('nephew', 'husband'): 0.00
('husband', 'niece'): 0.00
('husband', 'daughter'): 0.00


=====================daughter=====================
('sister', 'son'): 0.68
('sister', 'daughter'): 0.61
('daughter

### Micro saturation

In [13]:
for rel in micro_saturations:
    print(f"{id2rel[rel]:=^50}")
    sorted_items = sorted(micro_saturations[rel].items(), key=lambda x: x[1], reverse=True)
    for i, (path, saturation) in enumerate(sorted_items):
        if i == topk_micro:
            break
        print(f"{tuple(id2rel[r] for r in path)}: {saturation:.2f}")
    print("\n")

=======================aunt=======================
('sister', 'uncle'): 0.26
('sister', 'aunt'): 0.22
('aunt', 'brother'): 0.21
('aunt', 'sister'): 0.18
('sister', 'father'): 0.09
('sister', 'mother'): 0.05
('aunt', 'wife'): 0.00
('aunt', 'husband'): 0.00
('wife', 'uncle'): 0.00
('mother', 'wife'): 0.00


=====================brother======================
('brother', 'brother'): 0.14
('nephew', 'uncle'): 0.13
('brother', 'sister'): 0.13
('uncle', 'nephew'): 0.12
('nephew', 'aunt'): 0.11
('uncle', 'niece'): 0.09
('son', 'father'): 0.08
('son', 'mother'): 0.07
('father', 'nephew'): 0.04
('uncle', 'son'): 0.04


=====================daughter=====================
('sister', 'son'): 0.25
('sister', 'daughter'): 0.20
('daughter', 'husband'): 0.15
('daughter', 'wife'): 0.14
('niece', 'brother'): 0.10
('niece', 'sister'): 0.09
('wife', 'nephew'): 0.00
('wife', 'son'): 0.00
('daughter', 'niece'): 0.00
('niece', 'nephew'): 0.00


======================father======================
('husband', 'mo

### Comprehensive saturation

In [14]:
comp_saturations = {
    rel: {} for rel in micro_saturations
}

for rel in micro_saturations:
    for path, value in micro_saturations[rel].items():
        comp_saturations[rel][path] = value * macro_saturations[rel][path]

for rel in comp_saturations:
    print(f"{id2rel[rel]:=^50}")
    sorted_items = sorted(comp_saturations[rel].items(), key=lambda x: x[1], reverse=True)
    for i, (path, saturation) in enumerate(sorted_items):
        if i == topk_comp:
            break
        print(f"{tuple(id2rel[r] for r in path)}: {saturation:.2f}%")        
    print("\n")

=======================aunt=======================
('sister', 'uncle'): 0.23%
('sister', 'aunt'): 0.19%
('aunt', 'brother'): 0.17%
('aunt', 'sister'): 0.13%
('sister', 'father'): 0.06%
('sister', 'mother'): 0.02%
('aunt', 'wife'): 0.00%
('aunt', 'husband'): 0.00%
('wife', 'uncle'): 0.00%
('mother', 'wife'): 0.00%


=====================brother======================
('brother', 'brother'): 0.12%
('nephew', 'uncle'): 0.10%
('brother', 'sister'): 0.10%
('son', 'father'): 0.08%
('nephew', 'aunt'): 0.08%
('uncle', 'nephew'): 0.08%
('son', 'mother'): 0.07%
('uncle', 'niece'): 0.05%
('father', 'nephew'): 0.01%
('uncle', 'son'): 0.01%


=====================daughter=====================
('sister', 'son'): 0.17%
('sister', 'daughter'): 0.12%
('daughter', 'husband'): 0.07%
('daughter', 'wife'): 0.06%
('niece', 'brother'): 0.04%
('niece', 'sister'): 0.03%
('wife', 'nephew'): 0.00%
('wife', 'brother'): 0.00%
('niece', 'husband'): 0.00%
('daughter', 'niece'): 0.00%


======================father===